In [ ]:
import folium
import geopandas
import pandas as pd
from geopy.distance import geodesic
import time

In [3]:
url = "minimalist_coning.geojson"
DATA_URL = "Data/20230327.parquet"

start = time.time()
df = pd.read_parquet(path=DATA_URL, engine= "pyarrow")
print(f"Took: {time.time() - start} s ")

Took: 60.62552499771118 s 


In [ ]:
m = folium.Map([46, 14.5], zoom_start=9)

In [ ]:
print(df.columns)
display(df)
columns = ['deviceid', 'date', 'time', 'lon', 'lat']

Index(['deviceid', 'date', 'time', 'lon', 'lat'], dtype='object')


,deviceid,date,time,lon,lat
0,657192d6fb7684195ed9918cafc556bccb77df641ecba8...,27.03.2023,20:00:00,14.46000,46.074120
1,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,20:00:00,14.46000,46.074120
2,91dc945545a75dc878d90796d8471a4afbea1250b4b76f...,27.03.2023,20:00:00,14.22700,46.351559
3,9c7cf2f12d7a3fc42fceae30c880a49fdb918047d8a936...,27.03.2023,20:00:00,14.22700,46.351559
4,0b6df71eaf8374cea223359dde92425b59871c8bd46d05...,27.03.2023,20:00:00,14.22700,46.351559
...,...,...,...,...,...
133453150,3a3fbc843ac96dc4b3f023d361744d831797d1ad0becde...,27.03.2023,03:59:58,15.27919,46.250469
133453151,bd54ed4464deb0b5cbcba7d228b863bcf7cb79598567db...,27.03.2023,03:59:58,15.28093,46.244411
133453152,2957081f1f7c0f48997b3cc148a2662caaaf3be4858fb2...,27.03.2023,03:59:58,15.29142,46.141102
133453153,0543fd83d892befecf3bfd776d55c23fbd024195f10844...,27.03.2023,03:59:58,15.29445,46.007992


In [ ]:
one_device = df[df["deviceid"].str.contains("feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea")]

In [ ]:
sorted = one_device.sort_values(by="time")

sorted["date"] = sorted["date"].astype(str)
sorted["time"] = sorted["time"].astype(str)
sorted = sorted.reset_index(drop=True)
display(sorted)

,deviceid,date,time,lon,lat
0,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:00:14,14.47389,46.076820
1,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:00:56,14.47389,46.076820
2,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:07:38,14.47389,46.076820
3,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:10:47,14.47389,46.076820
4,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:10:58,14.47389,46.076820
...,...,...,...,...,...
925,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,23:24:54,14.46365,46.079472
926,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,23:25:08,14.45964,46.073860
927,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,23:25:17,14.47389,46.076820
928,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,23:25:27,14.51276,45.945541


In [ ]:
def add_line_from_to(initial:list[int, int], final:list[int, int], m:folium.Map):
    coord = [initial, final]
    fill = folium.vector_layers.path_options(fill=True, fill_color="#A00000")
    folium.PolyLine(
        locations=coord,
        color="#A00000",
        weight=5,
        tooltip="From start to finish"
    ).add_to(m)
    folium.RegularPolygonMarker(
        location=final,
        color="#A00000",
        number_of_sides=3, 
        radius=10, kwargs=fill).add_to(m)
    
Point_A = [46.25 , 15.30]
Point_B = [45.9, 14.216]
# add_line_from_to(Point_A, Point_B, m=m)

In [ ]:
#Creating Random Fake Data
import numpy as np
from datetime import datetime, timedelta
from geopy.distance import distance as geopy_distance
from geopy import Point

distances = []
time_deltas = []
speeds = []
used = []
display(sorted.head())
FMT = '%H:%M:%S'
for i in range(1, len(sorted)):
    point1 = (sorted.loc[i-1, "lat"], sorted.loc[i-1, "lon"])
    time1 = sorted.loc[i-1, "time"]
    point2 = (sorted.loc[i, "lat"], sorted.loc[i, "lon"])
    time2 = sorted.loc[i, "time"]
    distance = geodesic(point1, point2).meters
    time_delta = (datetime.strptime(time2, FMT) - datetime.strptime(time1, FMT)).seconds
    if time_delta==0:
        speed = 0
    else:
        speed = (distance/time_delta)*3.6
    
    if speed<200 and speed>1:
        distances.append(distance)
        time_deltas.append(time_delta)
        speeds.append(speed)
        used.append(i)
        add_line_from_to([sorted.loc[i-1, 'lat'],sorted.loc[i-1, 'lon']], [sorted.loc[i, 'lat'], sorted.loc[i, 'lon']], m=m)

# Add results to DataFrame
day_dict = {"distance": distances, "delta_t":time_deltas, "speeds (km/h)":time_deltas}
day_dict = pd.DataFrame(day_dict)
print(day_dict)


,deviceid,date,time,lon,lat
0,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:00:14,14.47389,46.07682
1,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:00:56,14.47389,46.07682
2,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:07:38,14.47389,46.07682
3,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:10:47,14.47389,46.07682
4,feea6e0a2b11c2b7d70ea4b239e309a1c48986b91becea...,27.03.2023,00:10:58,14.47389,46.07682


        distance  delta_t  speeds (km/h)
0    4800.547972      196            196
1    2613.082579       61             61
2    2279.036107       87             87
3     646.250826      298            298
4    1075.548788      190            190
..           ...      ...            ...
232   148.739018       28             28
233   453.870451       81             81
234   148.739018       33             33
235   503.432130      478            478
236   696.610188       14             14

[237 rows x 3 columns]


In [ ]:
m